# Scraping NCRS SNOTEL locations

beautifulsoup only works in python 3 for some reason

In [1]:
fmt = 'http://wcc.sc.egov.usda.gov/nwcc/yearcount?network=sntl&state={state}&counttype=statelist'

In [2]:
url = fmt.format(state='WA')
print(url)

http://wcc.sc.egov.usda.gov/nwcc/yearcount?network=sntl&state=WA&counttype=statelist


In [3]:
import requests

In [4]:
r = requests.get(url)

In [5]:
import bs4

In [6]:
soup = bs4.BeautifulSoup(r.text, 'lxml')
tables = soup.findAll('table')
results = tables[4]

In [7]:
import pandas

In [8]:
thing = pandas.read_html(str(results), header=0)
sites = thing[0]

In [9]:
site_id = sites.site_name.str.extract('\((\d+)\)')
site_id.name = 'site_id'
sites = sites.set_index(site_id)

In [10]:
sites

,ntwk,state,site_name,ts,start,lat,lon,elev,county,huc
site_id,,,,,,,,,,
908,SNTL,WA,Alpine Meadows (908),NaN,1994-September,47.78,-121.70,3500,King,Upper North Fork Tolt River (171100100501)
990,SNTL,WA,Beaver Pass (990),NaN,2001-October,48.88,-121.26,3630,Whatcom,McNaught Creek-Skagit River (171100050305)
352,SNTL,WA,Blewett Pass (352),NaN,1980-September,47.35,-120.68,4240,Chelan,Upper Peshastin Creek (170200110501)
1080,SNTL,WA,Brown Top (1080),NaN,2009-September,48.93,-121.20,5830,Whatcom,Maselpanik Creek (171100050301)
1107,SNTL,WA,Buckinghorse (1107),NaN,2008-September,47.71,-123.46,4870,Jefferson,Headwaters Elwha River (171100200501)
375,SNTL,WA,Bumping Ridge (375),NaN,1978-October,46.81,-121.33,4610,Yakima,Upper Bumping River (170300020105)
376,SNTL,WA,Bunchgrass Mdw (376),NaN,1980-August,48.69,-117.18,5000,Pend Oreille,East Branch Le Clerc Creek (170102160303)
942,SNTL,WA,Burnt Mountain (942),NaN,1999-July,47.04,-121.94,4170,Pierce,Wilkeson Creek (171100140103)
1109,SNTL,WA,Calamity (1109),NaN,2008-August,45.90,-122.22,2500,Skamania,Lower Canyon Creek (170800020603)


# plot the locations

In [11]:
locs = sites[['lat','lon']]

In [12]:
import gmaps

<IPython.core.display.Javascript object>

In [13]:
gmaps.heatmap(locs.values)